Import Library

In [ ]:
!pip install swifter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
import string
import re
import random as rd
import math

Load Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def load_data():
    data = pd.read_excel('/content/drive/MyDrive/KULIAH/PROGRAM/pinjol_new.xlsx')
    return data

In [ ]:
tweet_df = load_data()
tweet_df.head(5)

,Text,Id
0,RT @meyulians: Senilai Rp 217 miliar berhasil ...,1.461846e+18
1,RT @meyulians: Bagaikan mati satu tumbuh serib...,1.461846e+18
2,RT @meyulians: Dittipideksus Bareskrim Polri a...,1.461846e+18
3,RT @KaredokLeunca__: Uwoooww..ternyata kilang ...,1.461844e+18
4,Ini Daftar 151 Pinjol Ilegal yang Sudah 'Disap...,1.461843e+18


In [ ]:
dataframe = pd.DataFrame(tweet_df[['Id','Text']])
dataframe

,Id,Text
0,1.461846e+18,RT @meyulians: Senilai Rp 217 miliar berhasil ...
1,1.461846e+18,RT @meyulians: Bagaikan mati satu tumbuh serib...
2,1.461846e+18,RT @meyulians: Dittipideksus Bareskrim Polri a...
3,1.461844e+18,RT @KaredokLeunca__: Uwoooww..ternyata kilang ...
4,1.461843e+18,Ini Daftar 151 Pinjol Ilegal yang Sudah 'Disap...
...,...,...
6902,1.459867e+18,@idextratime @gbsans Pinjol FC
6903,1.459866e+18,@bertanyarl Yang terakhir diteror pinjol
6904,1.459865e+18,@tempodotco @DonAdam68 Bangsa yg begini mendap...
6905,1.459864e+18,@Drama_Alter Aku pinjol mboan modal off ni wak...


In [ ]:
dataframe.shape

(6907, 2)

Case Folding

In [ ]:
#case folding

tweet_df['Text'] = tweet_df['Text'].str.lower()

print('Case Folding Result : \n')
print(tweet_df['Text'].head(5))
print('\n\n')

Case Folding Result : 

0    rt @meyulians: senilai rp 217 miliar berhasil ...
1    rt @meyulians: bagaikan mati satu tumbuh serib...
2    rt @meyulians: dittipideksus bareskrim polri a...
3    rt @karedokleunca__: uwoooww..ternyata kilang ...
4    ini daftar 151 pinjol ilegal yang sudah 'disap...
Name: Text, dtype: object





Tokenizing

In [ ]:
# import word_tokenize & FreqDist from NLTK
nltk.download('punkt')

from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# -------TOKENIZING-----------------------
def remove_tweet_special(text):
    text = str(text)
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
tweet_df['Text'] = tweet_df['Text'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

tweet_df['Text'] = tweet_df['Text'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

tweet_df['Text'] = tweet_df['Text'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

tweet_df['Text'] = tweet_df['Text'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

tweet_df['Text'] = tweet_df['Text'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

tweet_df['Text'] = tweet_df['Text'].apply(remove_singl_char)

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

tweet_df['tweet_tokens'] = tweet_df['Text'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(tweet_df['tweet_tokens'].head())
print('\n\n\n')

Tokenizing Result : 

0    [rt, senilai, rp, miliar, berhasil, disita, ba...
1    [rt, bagaikan, mati, satu, tumbuh, seribu, pin...
2    [rt, dittipideksus, bareskrim, polri, akan, me...
3    [rt, uwooowwternyata, kilang, minyak, yg, terb...
4    [ini, daftar, pinjol, ilegal, yang, sudah, dis...
Name: tweet_tokens, dtype: object






In [ ]:
# NLTK calc frequency distribution
def FreqDist_wrapper(text):
  return FreqDist(text)

tweet_df['tweet_tokens_fdist'] = tweet_df['tweet_tokens'].apply(FreqDist_wrapper)

print('Frequency Tokens : \n')
print(tweet_df['tweet_tokens_fdist'].head().apply(lambda x : x.most_common()))

Frequency Tokens : 

0    [(rt, 1), (senilai, 1), (rp, 1), (miliar, 1), ...
1    [(sedikit, 2), (rt, 1), (bagaikan, 1), (mati, ...
2    [(rt, 1), (dittipideksus, 1), (bareskrim, 1), ...
3    [(rt, 1), (uwooowwternyata, 1), (kilang, 1), (...
4    [(ini, 1), (daftar, 1), (pinjol, 1), (ilegal, ...
Name: tweet_tokens_fdist, dtype: object


Filtering (Stopword Removal)

In [ ]:
nltk.download('stopwords')

from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#get stopwords Indonesia
list_stopwords = stopwords.words('indonesian')

# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])

# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
txt_stopword = pd.read_csv("/content/drive/MyDrive/KULIAH/PROGRAM/stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

tweet_df['tweet_tokens_WSW'] = tweet_df['tweet_tokens'].apply(stopwords_removal) 


print(tweet_df['tweet_tokens_WSW'].head())


0    [senilai, rp, miliar, berhasil, disita, baresk...
1    [mati, tumbuh, seribu, pinjol, illegal, punah,...
2    [dittipideksus, bareskrim, polri, mengenakan, ...
3    [uwooowwternyata, kilang, minyak, terbakar, ke...
4    [daftar, pinjol, ilegal, disapu, bersih, pemer...
Name: tweet_tokens_WSW, dtype: object


In [ ]:
token_without_stopword = tweet_df['tweet_tokens_WSW']
token_without_stopword

0       [senilai, rp, miliar, berhasil, disita, baresk...
1       [mati, tumbuh, seribu, pinjol, illegal, punah,...
2       [dittipideksus, bareskrim, polri, mengenakan, ...
3       [uwooowwternyata, kilang, minyak, terbakar, ke...
4       [daftar, pinjol, ilegal, disapu, bersih, pemer...
                              ...                        
6902                                         [pinjol, fc]
6903                                    [diteror, pinjol]
6904    [bangsa, prioritas, hati, lord, pemerintah, ju...
6905    [alter, pinjol, mboan, modal, off, ni, wakakakaa]
6906    [sy, sampe, risih, wa, telp, dc, gara, temen, ...
Name: tweet_tokens_WSW, Length: 6907, dtype: object

Normalisasi

In [ ]:
#NORMALIZATION
normalizad_word = pd.read_csv("/content/drive/MyDrive/KULIAH/PROGRAM/slang_words.txt")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

tweet_df['tweet_normalized'] = tweet_df['tweet_tokens_WSW'].apply(normalized_term)

tweet_df['tweet_normalized'].head(10)

0    [senilai, rp, miliar, berhasil, disita, baresk...
1    [mati, tumbuh, seribu, pinjol, illegal, punah,...
2    [dittipideksus, bareskrim, polri, mengenakan, ...
3    [uwooowwternyata, kilang, minyak, terbakar, ke...
4    [daftar, pinjol, ilegal, disapu, bersih, pemer...
5                [kawal, berantas, pinjol, ilegal, cc]
6           [anti, urusan, bank, ngutangnya, kepinjol]
7                                     [daftar, pinjol]
8    [semangat, berantas, pinjol, ilegal, pa, teten...
9         [kek, pinjol, ok, suka, nagih, ngirim, foto]
Name: tweet_normalized, dtype: object

In [ ]:
tweet_df.head(3)

,Text,Id,tweet_tokens,tweet_tokens_WSW,tweet_normalized
0,rt senilai rp miliar berhasil disita bareskrim...,1.461846e+18,"[rt, senilai, rp, miliar, berhasil, disita, ba...","[senilai, rp, miliar, berhasil, disita, baresk...","[senilai, rp, miliar, berhasil, disita, baresk..."
1,rt bagaikan mati satu tumbuh seribu pinjol ill...,1.461846e+18,"[rt, bagaikan, mati, satu, tumbuh, seribu, pin...","[mati, tumbuh, seribu, pinjol, illegal, punah,...","[mati, tumbuh, seribu, pinjol, illegal, punah,..."
2,rt dittipideksus bareskrim polri akan mengenak...,1.461846e+18,"[rt, dittipideksus, bareskrim, polri, akan, me...","[dittipideksus, bareskrim, polri, mengenakan, ...","[dittipideksus, bareskrim, polri, mengenakan, ..."


Stemmer

In [ ]:
!pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import Sastrawi package

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

In [ ]:
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in tweet_df['tweet_normalized']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

tweet_df['tweet_tokens_stemmed'] = tweet_df['tweet_normalized'].swifter.apply(get_stemmed_term)
print(tweet_df['tweet_tokens_stemmed'])

Output streaming akan dipotong hingga 5000 baris terakhir.
berhubungan : hubung
smua : smua
trmasuk : trmasuk
plecit : plecit
tekni : tekni
ter : ter
brantas : brantas
akarnya : akar
cintanda : cintanda
berkedok : kedok
jaksel : jaksel
slalu : slalu
yorobun : yorobun
kesebar : sebar
ayam : ayam
berapis : rap
wamena : wamena
terapkan : terap
dipakein : dipakein
pinjolkesel : pinjolkesel
tlpon : tlpon
sma : sma
pinjoludh : pinjoludh
dibilang : bilang
msih : msih
tangkas : tangkas
hujan : hujan
angin : angin
bantuannya : bantu
menganggu : menganggu
ngebantu : ngebantu
kepepet : pepet
pokoknya : pokok
akunnya : akun
illegaldittipideksus : illegaldittipideksus
kerenn : kerenn
aplikasinya : aplikasi
request : request
keren : keren
kgk : kgk
terimakasih : terimakasih
urung : urung
bencana : bencana
banjir : banjir
sintang : sintang
disalahkan : salah
kerusakan : rusa
area : area
tangkapan : tangkap
uu : uu
kpk : kpk
omnibuslaw : omnibuslaw
minerba : minerba
pajek : pajek
ln : ln
bansos : bans

Pandas Apply:   0%|          | 0/6907 [00:00<?, ?it/s]

0       [nila, rp, miliar, hasil, sita, bareskrim, pol...
1       [mati, tumbuh, ribu, pinjol, illegal, punah, p...
2       [dittipideksus, bareskrim, polri, kena, wna, b...
3       [uwooowwternyata, kilang, minyak, bakar, kemar...
4        [daftar, pinjol, ilegal, sapu, bersih, perintah]
                              ...                        
6902                                         [pinjol, fc]
6903                                      [teror, pinjol]
6904    [bangsa, prioritas, hati, lord, perintah, jubi...
6905    [alter, pinjol, mboan, modal, off, ni, wakakakaa]
6906    [sy, sampe, risih, wa, telp, dc, gara, temen, ...
Name: tweet_tokens_stemmed, Length: 6907, dtype: object


In [ ]:
tweet_df

0       [nila, rp, miliar, hasil, sita, bareskrim, pol...
1       [mati, tumbuh, ribu, pinjol, illegal, punah, p...
2       [dittipideksus, bareskrim, polri, kena, wna, b...
3       [uwooowwternyata, kilang, minyak, bakar, kemar...
4        [daftar, pinjol, ilegal, sapu, bersih, perintah]
                              ...                        
6902                                         [pinjol, fc]
6903                                      [teror, pinjol]
6904    [bangsa, prioritas, hati, lord, perintah, jubi...
6905    [alter, pinjol, mboan, modal, off, ni, wakakakaa]
6906    [sy, sampe, risih, wa, telp, dc, gara, temen, ...
Name: tweet_tokens_stemmed, Length: 6907, dtype: object

Save Prepocessing Data

In [ ]:
tweet_df.to_csv("Text_Prepocessing.csv")

In [ ]:
from google.colab import files
files.download('Text_Prepocessing.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
tweet_df.to_excel("prepo_new.xlsx")

In [ ]:
from google.colab import files
files.download('prepo_new.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
tweet_df.head(5)

,Text,Id,tweet_tokens,tweet_tokens_fdist,tweet_tokens_WSW,tweet_normalized,tweet_tokens_stemmed,Label
0,rt senilai rp miliar berhasil disita bareskrim...,1461846164626490112,"[rt, senilai, rp, miliar, berhasil, disita, ba...","{'rt': 1, 'senilai': 1, 'rp': 1, 'miliar': 1, ...","[senilai, rp, miliar, berhasil, disita, baresk...","[senilai, rp, miliar, berhasil, disita, baresk...","[nila, rp, miliar, hasil, sita, bareskrim, pol...",-0.5
1,rt bagaikan mati satu tumbuh seribu pinjol ill...,1461846151737469952,"[rt, bagaikan, mati, satu, tumbuh, seribu, pin...","{'rt': 1, 'bagaikan': 1, 'mati': 1, 'satu': 1,...","[mati, tumbuh, seribu, pinjol, illegal, punah,...","[mati, tumbuh, seribu, pinjol, illegal, punah,...","[mati, tumbuh, ribu, pinjol, illegal, punah, p...",-0.5
2,rt dittipideksus bareskrim polri akan mengenak...,1461846125170739968,"[rt, dittipideksus, bareskrim, polri, akan, me...","{'rt': 1, 'dittipideksus': 1, 'bareskrim': 1, ...","[dittipideksus, bareskrim, polri, mengenakan, ...","[dittipideksus, bareskrim, polri, mengenakan, ...","[dittipideksus, bareskrim, polri, kena, wna, b...",-0.5
3,rt uwooowwternyata kilang minyak yg terbakar k...,1461844086239869952,"[rt, uwooowwternyata, kilang, minyak, yg, terb...","{'rt': 1, 'uwooowwternyata': 1, 'kilang': 1, '...","[uwooowwternyata, kilang, minyak, terbakar, ke...","[uwooowwternyata, kilang, minyak, terbakar, ke...","[uwooowwternyata, kilang, minyak, bakar, kemar...",0.0
4,ini daftar pinjol ilegal yang sudah disapu ber...,1461842864116100096,"[ini, daftar, pinjol, ilegal, yang, sudah, dis...","{'ini': 1, 'daftar': 1, 'pinjol': 1, 'ilegal':...","[daftar, pinjol, ilegal, disapu, bersih, pemer...","[daftar, pinjol, ilegal, disapu, bersih, pemer...","[daftar, pinjol, ilegal, sapu, bersih, perintah]",0.0


Labeling NaiveBayesAnalyzer

EXPORT DATASET LABELING

In [ ]:
df.to_csv("pinjol_labeling.csv")

In [ ]:
df.to_excel("pinjol_labeling.xlsx")

In [ ]:
from google.colab import files
files.download('pinjol_labeling.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('pinjol_labeling.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>